In [4]:
!bash ./start_vllm_container.sh

682fed21b0c09df2ae8d8afd84987a8156d9204dbf35782f333ebdc222aa8980
docker: Error response from daemon: driver failed programming external connectivity on endpoint inspiring_stonebraker (1e38955ba096c4fc0c021e3ece3f3ab326638d8f235c332a1b10cadcf9e582d2): Bind for 0.0.0.0:8000 failed: port is already allocated.


In [6]:
!pip install openai==0.28.1

  Obtaining dependency information for openai from https://files.pythonhosted.org/packages/1e/9f/385c25502f437686e4aa715969e5eaf5c2cb5e5ffa7c5cdd52f3c6ae967a/openai-0.28.1-py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.0/77.0 kB 3.3 MB/s eta 0:00:00


In [7]:
import openai

def invoke_vllm(prompt, assist_role=True, stream=True, max_tokens=400):

    # Modify OpenAI's API key and API base to use vLLM's API server.
    openai.api_key = "EMPTY"
    vllm_container_port = 8000
    openai.api_base = f"http://localhost:{vllm_container_port}/v1"

    # List models API
    models = openai.Model.list()
    #print("Models:", models)

    model = models["data"][0]["id"]

    # Completion API
    
    if (assist_role):
        prompt = f"<|prompter|>{prompt}</s><|assistant|>"

    completion = openai.Completion.create(
        model=model,
        prompt=prompt,
        echo=False,
        n=1,
        stream=stream,
        max_tokens=max_tokens,
        #ignore_eos=False,
        #temperature=0.1,
        temperature=0,
        #top_p=0.95,
        #stop=["</s>"],
     )
    if stream:
        output = ""
        for c in completion:
            #print(c)
            d = c["choices"][0]["text"]
            output += d
            print(d, end='', flush=True)      
    else:
        output = completion["choices"][0]["text"]
        print(output)
    return output

In [8]:
prompt = "What are the main challenges to support a long context for LLM?"
result = invoke_vllm(prompt)

 There are several challenges to supporting a long context for LLMs, including:

1. Memory Limitations: LLMs are limited in the amount of data they can store in memory, which can limit the length of the context they can support.

2. Computational Complexity: Generating responses for long contexts can be computationally expensive, especially for larger models.

3. Training Data: Training data for long contexts can be difficult to obtain, as it requires a large amount of text data that is relevant and coherent.

4. Bias and Unanswerable Questions: Long contexts can increase the likelihood of bias and unanswerable questions, as the model may not have seen similar contexts during training.

5. Incomplete or Inconsistent Data: Long contexts can be more susceptible to errors and inconsistencies in the data, which can affect the accuracy of the model's responses.

6. Maintenance and Updating: Maintaining and updating a long context can be challenging, as it requires regular monitoring and upd

In [9]:
with open("../example_long_ctx.txt", "r") as fin:
    task_instruction = fin.read()
    task_instruction = task_instruction.format(
        my_question="please tell me how does pgvector help with Generative AI and give me some examples."
    )
prompt = f"<|prompter|>{task_instruction}</s><|assistant|>"
invoke_vllm(prompt)

 pgvector is an open-source extension for PostgreSQL supported by Amazon Aurora PostgreSQL-Compatible Edition.

You can use pgvector to store, search, index, and query billions of embeddings that are generated from machine learning (ML) and artificial intelligence (AI) models in your database, such as those from Amazon Bedrock (limited preview) or Amazon SageMaker. A vector embedding is a numerical representation that represents the semantic meaning of content such as text, images, and video.

With pgvector, you can query embeddings in your Aurora PostgreSQL database to perform efficient semantic similarity searches of these data types, represented as vectors, combined with other tabular data in Aurora. This enables the use of generative AI and other AI/ML systems for new types of applications such as personalized recommendations based on similar text descriptions or images, candidate match based on interview notes, customer service next best action recommendations based on successful 

' pgvector is an open-source extension for PostgreSQL supported by Amazon Aurora PostgreSQL-Compatible Edition.\n\nYou can use pgvector to store, search, index, and query billions of embeddings that are generated from machine learning (ML) and artificial intelligence (AI) models in your database, such as those from Amazon Bedrock (limited preview) or Amazon SageMaker. A vector embedding is a numerical representation that represents the semantic meaning of content such as text, images, and video.\n\nWith pgvector, you can query embeddings in your Aurora PostgreSQL database to perform efficient semantic similarity searches of these data types, represented as vectors, combined with other tabular data in Aurora. This enables the use of generative AI and other AI/ML systems for new types of applications such as personalized recommendations based on similar text descriptions or images, candidate match based on interview notes, customer service next best action recommendations based on succes